# MERGE005
Best of Each Type as of July 3

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [2]:
tracking = pd.read_csv('../tracking/tracking.csv')

In [3]:
tracking.head()

,run_id,model_number,eval_metric,n_estimators,early_stopping_rounds,random_state,n_threads,learning_rate,n_fold,model_type,training_time,n_features,depth,gml_oof_score,oof_score,lb_score,1JHC_mae_cv,2JHH_mae_cv,1JHN_mae_cv,2JHN_mae_cv,2JHC_mae_cv,3JHH_mae_cv,3JHC_mae_cv,3JHN_mae_cv,1JHC_std_mae_cv,2JHH_std_mae_cv,1JHN_std_mae_cv,2JHN_std_mae_cv,2JHC_std_mae_cv,3JHH_std_mae_cv,3JHC_std_mae_cv,3JHN_std_mae_cv,1JHC_tr_sec_f1,1JHC_pred_sec_f1,1JHC_tr_sec_f2,1JHC_pred_sec_f2,2JHH_tr_sec_f1,2JHH_pred_sec_f1,2JHH_tr_sec_f2,2JHH_pred_sec_f2,1JHN_tr_sec_f1,1JHN_pred_sec_f1,1JHN_tr_sec_f2,1JHN_pred_sec_f2,2JHN_tr_sec_f1,2JHN_pred_sec_f1,2JHN_tr_sec_f2,2JHN_pred_sec_f2,2JHC_tr_sec_f1,2JHC_pred_sec_f1,2JHC_tr_sec_f2,2JHC_pred_sec_f2,3JHH_tr_sec_f1,3JHH_pred_sec_f1,3JHH_tr_sec_f2,3JHH_pred_sec_f2,3JHC_tr_sec_f1,3JHC_pred_sec_f1,3JHC_tr_sec_f2,3JHC_pred_sec_f2,3JHN_tr_sec_f1,3JHN_pred_sec_f1,3JHN_tr_sec_f2,3JHN_pred_sec_f2,1JHC_csv_save_sec,2JHH_csv_save_sec,1JHN_csv_save_sec,2JHN_csv_save_sec,2JHC_csv_save_sec,3JHH_csv_save_sec,3JHC_csv_save_sec,3JHN_csv_save_sec,1JHC_tr_sec_f3,1JHC_pred_sec_f3,2JHH_tr_sec_f3,2JHH_pred_sec_f3,1JHN_tr_sec_f3,1JHN_pred_sec_f3,2JHN_tr_sec_f3,2JHN_pred_sec_f3,3JHH_tr_sec_f3,3JHH_pred_sec_f3,2JHC_tr_sec_f3,2JHC_pred_sec_f3,3JHC_tr_sec_f3,3JHC_pred_sec_f3,3JHN_tr_sec_f3,3JHN_pred_sec_f3,2JHN_f1_best_iter,2JHN_f2_best_iter,1JHN_f1_best_iter,1JHN_f2_best_iter,3JHN_f1_best_iter,3JHN_f2_best_iter,2JHH_f1_best_iter,2JHH_f2_best_iter,3JHH_f1_best_iter,3JHH_f2_best_iter,1JHC_f1_best_iter,1JHC_f2_best_iter,2JHC_f1_best_iter,2JHC_f2_best_iter,3JHC_f1_best_iter,3JHC_f2_best_iter
0,0610_1341,M010,mae,500000.0,500.0,529.0,32.0,0.5,5.0,NaN,NaN,61.0,NaN,NaN,0.9210,-0.452,1.9206,0.4542,1.0336,0.5335,0.8405,0.5699,0.8593,0.3264,0.00388,0.00112,0.00945,0.00394,0.00131,0.00149,0.00279,0.00184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0611_0308,M012,mae,500000.0,500.0,529.0,32.0,0.5,5.0,NaN,NaN,28.0,NaN,NaN,1.0109,-0.202,1.8632,0.4782,0.9773,0.5826,1.0352,0.6167,0.9867,0.3553,0.00415,0.00060,0.01524,0.00399,0.00435,0.00285,0.00162,0.00088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0611_1250,M013,mae,50000.0,500.0,529.0,32.0,0.5,5.0,catboost,137353.19,28.0,NaN,NaN,0.9488,NaN,1.7478,0.4442,1.0337,0.5300,0.9603,0.5761,0.9359,0.3300,0.00485,0.00293,0.01892,0.00264,0.00330,0.00167,0.00182,0.00202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0611_1249,M014,mae,500000.0,500.0,529.0,24.0,0.3,5.0,NaN,NaN,28.0,NaN,NaN,0.9142,-0.275,1.7005,0.4378,0.8751,0.5070,0.9209,0.5596,0.8967,0.3167,0.00432,0.00156,0.01019,0.00536,0.00292,0.00243,0.00197,0.00191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0613_0402,M015,mae,500000.0,500.0,529.0,24.0,0.3,5.0,NaN,NaN,51.0,NaN,NaN,0.7914,-0.671,1.6918,0.3843,0.9161,0.4388,0.7155,0.4726,0.7331,0.2789,0.00215,0.00168,0.00587,0.00425,0.00230,0.00194,0.00083,0.00063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [12]:
tracking[['model_number','run_id','model_type',
          '1JHC_mae_cv', '2JHH_mae_cv', '1JHN_mae_cv',
          '2JHN_mae_cv', '2JHC_mae_cv', '3JHH_mae_cv',
          '3JHC_mae_cv', '3JHN_mae_cv']].sort_values('3JHN_mae_cv').head()

,model_number,run_id,model_type,1JHC_mae_cv,2JHH_mae_cv,1JHN_mae_cv,2JHN_mae_cv,2JHC_mae_cv,3JHH_mae_cv,3JHC_mae_cv,3JHN_mae_cv
28,M037,0706_2029,catboost,0.5959,0.1440,0.4293,0.1419,0.2255,0.1537,0.2650,0.1176
13,M025,0629_1311,catboost,0.6691,0.1640,0.4348,0.1398,0.3002,0.1934,0.3905,0.1627
22,M032,0703_2133,catboost,0.6068,0.1615,0.4427,0.1406,0.2866,0.1990,0.3838,0.1647
27,M036,0705_2317,catboost,0.6143,0.1499,0.3872,0.1454,0.2771,0.1828,0.3759,0.1670
12,M024,0627_1434,lgbm,0.7073,0.1739,0.3909,0.1478,0.3198,0.2082,0.4221,0.1750


In [15]:
# 1JHC - M031
# 2JHH - M037
# 1JHN - M030
# 2JHN - M031
# 2JHC - M037
# 3JHH - M037
# 3JHC - M037
# 3JHN - M037

sub_M031 = pd.read_csv('../temp/M031/temp4of8_M031_0702_0822_submission_lgb_3folds_5000000iter_0.05lr.csv')
oof_M031 = pd.read_csv('../temp/M031/temp4of8_M031_0702_0822_oof_lgb_3folds_5000000iter_0.05lr.csv')

In [16]:
sub_M037 = pd.read_csv('../submissions/M037_0706_2029_submission_lgb_2folds_-1.5106CV_500000iter_0.1lr.csv')
oof_M037 = pd.read_csv('../oof/M037_0706_2029_oof_catboost_2folds_-1.5106CV_500000iter_0.1lr.csv')

In [18]:
sub_M030_a = pd.read_csv('../temp/M030/temp4of8_M030_0701_1300_submission_lgb_3folds_5000000iter_0.05lr.csv')
sub_M030_b = pd.read_csv('../temp/M030/temp4of8_M030_0701_1302_submission_lgb_3folds_5000000iter_0.05lr.csv')
sub_M030_c = pd.read_csv('../temp/M030/temp4of8_M030_0701_1304_submission_lgb_3folds_5000000iter_0.05lr.csv')
sub_M030 = sub_M030_a.copy()
sub_M030['scc_a'] = sub_M030_a['scalar_coupling_constant']
sub_M030['scc_b'] = sub_M030_b['scalar_coupling_constant']
sub_M030['scc_c'] = sub_M030_c['scalar_coupling_constant']
sub_M030['scalar_coupling_constant'] = sub_M030[['scc_a','scc_b','scc_c']].mean(axis=1)
sub_M030 = sub_M030.drop(['scc_a','scc_b','scc_c'], axis=1).copy()

In [19]:
oof_M030_a = pd.read_csv('../temp/M030/temp4of8_M030_0701_1300_oof_lgb_3folds_5000000iter_0.05lr.csv')
oof_M030_b = pd.read_csv('../temp/M030/temp4of8_M030_0701_1302_oof_lgb_3folds_5000000iter_0.05lr.csv')
oof_M030_c = pd.read_csv('../temp/M030/temp4of8_M030_0701_1304_oof_lgb_3folds_5000000iter_0.05lr.csv')
oof_M030 = oof_M030_a.copy()
oof_M030['scc_a'] = oof_M030_a['oof_preds']
oof_M030['scc_b'] = oof_M030_b['oof_preds']
oof_M030['scc_c'] = oof_M030_c['oof_preds']
oof_M030['oof_preds'] = oof_M030[['scc_a','scc_b','scc_c']].sum(axis=1)
oof_M030 = oof_M030.drop(['scc_a','scc_b','scc_c'], axis=1).copy()

In [20]:
oof_final = oof_M031[['id','type','scalar_coupling_constant']].copy()
test = pd.read_csv('../input/test.csv')

oof_final.loc[oof_final['type'].isin(['1JHC','2JHN']), 'oof_preds'] = \
    oof_M031.loc[oof_final['type'].isin(['1JHC','2JHN'])]['oof_preds']
test.loc[test['type'].isin(['1JHC','2JHN']), 'scalar_coupling_constant'] = \
    sub_M031.loc[test['type'].isin(['1JHC','2JHN'])]['scalar_coupling_constant']

oof_final.loc[oof_final['type'].isin(['1JHN']), 'oof_preds'] = \
    oof_M030.loc[oof_final['type'].isin(['1JHN'])]['oof_preds']
test.loc[test['type'].isin(['1JHN']), 'scalar_coupling_constant'] = \
    sub_M030.loc[test['type'].isin(['1JHN'])]['scalar_coupling_constant']

oof_final.loc[oof_final['type'].isin(['2JHH','2JHC','3JHH','3JHC','3JHN']), 'oof_preds'] = \
    oof_M037.loc[oof_final['type'].isin(['2JHH','2JHC','3JHH','3JHC','3JHN'])]['oof_preds']
test.loc[test['type'].isin(['2JHH','2JHC','3JHH','3JHC','3JHN']), 'scalar_coupling_constant'] = \
    sub_M037.loc[test['type'].isin(['2JHH','2JHC','3JHH','3JHC','3JHN'])]['scalar_coupling_constant']

In [21]:
for i, d in oof_final.groupby('type'):
    print(i)
    print(mean_absolute_error(d['scalar_coupling_constant'], d['oof_preds']))

1JHC
0.5792659784491359
1JHN
0.35815790678334736
2JHC
0.22547268838617024
2JHH
0.14401644449312442
2JHN
0.13000911360249118
3JHC
0.265019456208353
3JHH
0.15371567574639028
3JHN
0.11761083550202311


In [22]:
group_mean_log_mae(oof_final['scalar_coupling_constant'], oof_final['oof_preds'], oof_final['type'])

-1.5476607984895852

In [24]:
sub_M025 = pd.read_csv('../submissions/M025_0629_1311_submission_lgb_2folds_-1.3265CV_500000iter_0.1lr.csv')
test['scc_good'] = sub_M025['scalar_coupling_constant']

In [25]:
test[['scc_good','scalar_coupling_constant']].corr()

,scc_good,scalar_coupling_constant
scc_good,1.000000,0.999938
scalar_coupling_constant,0.999938,1.000000


In [30]:
test['molecule_name']

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,scc_good
0,4658147,dsgdb9nsd_000004,2,0,2JHC,13.884890,8.873624
1,4658148,dsgdb9nsd_000004,2,1,1JHC,133.055422,134.914493
2,4658149,dsgdb9nsd_000004,2,3,3JHH,7.481856,3.716820
3,4658150,dsgdb9nsd_000004,3,0,1JHC,133.055422,134.914493
4,4658151,dsgdb9nsd_000004,3,1,2JHC,14.087441,8.873624
5,4658152,dsgdb9nsd_000015,3,0,1JHC,91.930736,93.465049
6,4658153,dsgdb9nsd_000015,3,2,3JHC,2.715360,2.231492
7,4658154,dsgdb9nsd_000015,3,4,2JHH,-7.536011,-7.838592
8,4658155,dsgdb9nsd_000015,3,5,2JHH,-9.475817,-9.731328
9,4658156,dsgdb9nsd_000015,4,0,1JHC,91.848660,93.010161


In [27]:
test[['id','scalar_coupling_constant']].to_csv('../submissions/MERGE007-M030_M031_M037_-1.5476CV.csv', index=False)

In [28]:
oof_final.to_csv('../oof/MERGE007-M030_M031_M037_-1.5476CV.csv', index=False)